In [1]:
# Import the os package, used to set env variables and check paths
import os
# import the dotenv package for client keys in .env file
from dotenv import load_dotenv
# for random UUID keys
# https://docs.python.org/3/library/uuid.html
import uuid
# Import square 
from square.client import Client
# for pretty print
import pprint
# for datetime stamp
import datetime

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Construct the .env file path
env_path = os.path.join(cwd, '.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [3]:
%%bash
#echo "SQUARE_ACCESS_TOKEN: ${SQUARE_ACCESS_TOKEN}"
#echo "SQUARE_APP_ID: ${SQUARE_APP_ID}"
#git clone https://github.com/square/square-python-sdk.git
#cd square-python-sdk
#pip install .


@hidden_cell

In [4]:
# init the square API
# using generic os environment this fails.
# However, when using the dotenv package it works
sq_client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production'  
)
pp = pprint.PrettyPrinter(indent=4)


# Create a new catalog item and associate with a new image

In terms of the Square API

* A catalog item example would be *t-shirt*
* A catalog item variation would be *size large t-shirt"

## step 1 create a catalog item and add to library

Everytime this is called, it creates a new one in the library.

Use `delete_catalog_object()` to delete them based upon obj_id

In [197]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)

# Specify the Catalog Item
# NOTE: The first catalog item requires one instance of a variation
item_name="Action Figure"
item_abbreviation = "ACFIGURE"
item_description = "something we want vertex ai to generate"
# item variation data includes
# description and price in us cents
item_variation_name = "Dr Doom Action Figure"
item_variation_amount = 375




result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": idempotency_key,
    "object": {
      "type": "ITEM",
      "id": "#thisvaries1",
      "item_data": {
        "name": item_name,
        "description": item_description,
        "abbreviation": item_abbreviation,
        "variations": [
          {
            "type": "ITEM_VARIATION",
            "id": "#thisvaries2",
            "item_variation_data": {
              "item_id": "#thisvaries1",
              "name": item_variation_name,
              "pricing_type": "FIXED_PRICING",
              "price_money": {
                "amount": item_variation_amount,
                "currency": "USD"
              }
            }
          },
        ]
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'catalog_object': {'type': 'ITEM', 'id': '7Z2R6MUWTYFGQ3K5PF7NL2WZ', 'updated_at': '2023-10-01T15:55:20.866Z', 'created_at': '2023-10-01T15:55:20.866Z', 'version': 1696175720866, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Action Figure', 'description': 'something we want vertex ai to generate', 'abbreviation': 'ACFIGURE', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'FM7JRSIOFQ3D3BZGKM7BJZQG', 'updated_at': '2023-10-01T15:55:20.866Z', 'created_at': '2023-10-01T15:55:20.866Z', 'version': 1696175720866, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '7Z2R6MUWTYFGQ3K5PF7NL2WZ', 'name': 'Dr Doom Action Figure', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 375, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}], 'product_type': 'REGULAR', 'description_html': '<p>something we want vertex ai to generate</p>', 'description_plaintext': 'something we want vert

In [198]:
# print and save the object id
pp.pprint(result.body)

# Save the id
#cat_obj_item_id = result.body['catalog_object']['id']
#print(cat_obj_item_id)

{   'catalog_object': {   'created_at': '2023-10-01T15:55:20.866Z',
                          'id': '7Z2R6MUWTYFGQ3K5PF7NL2WZ',
                          'is_deleted': False,
                          'item_data': {   'abbreviation': 'ACFIGURE',
                                           'description': 'something we want '
                                                          'vertex ai to '
                                                          'generate',
                                           'description_html': '<p>something '
                                                               'we want vertex '
                                                               'ai to '
                                                               'generate</p>',
                                           'description_plaintext': 'something '
                                                                    'we want '
                                                            

In [199]:
# print and save the object id
#pp.pprint(result.body)

#
# Save the id's for catalog and variation id
#
# Note this is the catalog item id.
# Later we want to add a picture for the item variation id.
cat_obj_item_id = result.body['catalog_object']['id']
#print(cat_obj_item_id)

cat_obj_item_variations = result.body['catalog_object']['item_data']['variations']
#pp.pprint(cat_obj_item_variations)


for x in  cat_obj_item_variations:
    #pp.pprint(x)
    #pp.pprint(x['item_variation_data']['item_id'])
    if item_variation_name == x['item_variation_data']['name']:
        #pp.pprint(x)
        cat_obj_item_variation_id = x['id']
        #pp.pprint(cat_obj_item_variation_id)

print("cat_obj_item_id : ", cat_obj_item_id )
print("cat_obj_item_variation_id: ", cat_obj_item_variation_id)
# Save off this variation id so we can use to update inventory
dr_doom_variation_id=cat_obj_item_variation_id


# save for later
#item_ids = [e["item_variations"]["item_id"] for e in cat_obj_item_id]


cat_obj_item_id :  7Z2R6MUWTYFGQ3K5PF7NL2WZ
cat_obj_item_variation_id:  FM7JRSIOFQ3D3BZGKM7BJZQG


## Step 2 upload an image an associate with the catalog item

Add an image to the catablog item variation.  In the case of a *large T-Shirt*, this would be an image of a Large T-Shirt.

In [200]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)


# DR DOOM
file_to_upload_path = "./sample_imgs/drdoom.jpeg" # Modify this to point to your desired file.
image_name = "Dr Doom Doll Image"
image_caption = "Dr Doom Doll"

f_stream = open(file_to_upload_path, "rb")



result = sq_client.catalog.create_catalog_image(
  request = {
    "idempotency_key": idempotency_key,
     "object_id": cat_obj_item_variation_id,
    "image": {
      "type": "IMAGE",
      "id": "#image_id",
      "image_data": {
        "name": image_name,
        "caption": image_caption
      }
    }
  },
  image_file = f_stream
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)


{'image': {'type': 'IMAGE', 'id': 'XEVPMNJIODMCNALOSSQFOY4Q', 'updated_at': '2023-10-01T15:55:30.698Z', 'created_at': '2023-10-01T15:55:30.698Z', 'version': 1696175730698, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'Dr Doom Doll Image', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/0a980973e5af640e65bc451ff2f4c565a2819d9e/original.jpeg', 'caption': 'Dr Doom Doll'}}}


# Create another variation for the same catalog item


In [201]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
#print("idempotency_key: ", idempotency_key)

# Specify the Catalog Item
# NOTE: The first catalog item requires one instance of a variation
#item_name="Action Figure"
#item_abbreviation = "ACFIGURE"
#item_description = "something we want vertex ai to generate"
# item variation data includes
# description and price in us cents
item_variation_name = "Nick Fury Action Figure"
item_variation_amount = 350




result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": idempotency_key,
    "object": {
      "type": "ITEM_VARIATION",
      "id": "#new",
      "item_variation_data": {
        "item_id": cat_obj_item_id,
        "name": item_variation_name,
        "pricing_type": "FIXED_PRICING",
        "price_money": {
          "amount": item_variation_amount,
          "currency": "USD"
        }
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)







{'catalog_object': {'type': 'ITEM_VARIATION', 'id': '2MD7GTL3C6H2KNWUNDJWKRJQ', 'updated_at': '2023-10-01T15:55:35.008Z', 'created_at': '2023-10-01T15:55:35.008Z', 'version': 1696175735008, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '7Z2R6MUWTYFGQ3K5PF7NL2WZ', 'name': 'Nick Fury Action Figure', 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}, 'id_mappings': [{'client_object_id': '#new', 'object_id': '2MD7GTL3C6H2KNWUNDJWKRJQ'}]}


In [202]:
# print and save the object id
pp.pprint(result.body)

#
# Save the id's for catalog and variation id
#
# Note this is the catalog item id.
# Later we want to add a picture for the item variation id.
# NOTE: I was expecting to have to search through the variations
# to find the variation id, when you have two items, but it appears you get just the last item
# so we don't have to search.  We know where it is.
cat_obj_item_variation_id = result.body['catalog_object']['id']

print(cat_obj_item_variation_id)

# Save off this variation id so we can use to update inventory
nick_fury_variation_id=cat_obj_item_variation_id



{   'catalog_object': {   'created_at': '2023-10-01T15:55:35.008Z',
                          'id': '2MD7GTL3C6H2KNWUNDJWKRJQ',
                          'is_deleted': False,
                          'item_variation_data': {   'item_id': '7Z2R6MUWTYFGQ3K5PF7NL2WZ',
                                                     'name': 'Nick Fury Action '
                                                             'Figure',
                                                     'price_money': {   'amount': 350,
                                                                        'currency': 'USD'},
                                                     'pricing_type': 'FIXED_PRICING',
                                                     'sellable': True,
                                                     'stockable': True},
                          'present_at_all_locations': True,
                          'type': 'ITEM_VARIATION',
                          'updated_at': '2023-10-01T15:55:35

# Add an image for the variation

In [203]:
# create a key
uniq_uuid = uuid.uuid4()
# convert to string
idempotency_key = str(uniq_uuid)
#print("uniq_uuid: ", uniq_uuid)
print("idempotency_key: ", idempotency_key)


# Nick Fury
file_to_upload_path = "./sample_imgs/nickfury.jpeg" # Modify this to point to your desired file.
image_name = "Nick Fury Doll Image"
image_caption = "Nick Fury Doll"

f_stream = open(file_to_upload_path, "rb")

result = sq_client.catalog.create_catalog_image(
   request = {
     "idempotency_key": idempotency_key,
      "object_id": cat_obj_item_variation_id,
     "image": {
       "type": "IMAGE",
       "id": "#image_id",
       "image_data": {
         "name": image_name,
         "caption": image_caption
       }
     }
   },
   image_file = f_stream
 )

if result.is_success():
   print(result.body)
elif result.is_error():
   print(result.errors)


idempotency_key:  701c9c11-0f65-4e29-8613-9bd2b802a8ab
{'image': {'type': 'IMAGE', 'id': '4BZDTISFPGBIVXQQWAWHTDB4', 'updated_at': '2023-10-01T15:55:39.874Z', 'created_at': '2023-10-01T15:55:39.874Z', 'version': 1696175739874, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'Nick Fury Doll Image', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/2e7be50da564218c0327c39300dfd2198445bd2d/original.jpeg', 'caption': 'Nick Fury Doll'}}}


# verify the catalog has the item we added

## Query all items and look through results manually

In [180]:
result = sq_client.catalog.list_catalog()

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)



{'objects': [{'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'TN6I6F7QXTEKDRT4M5F6JA7J', 'updated_at': '2023-09-17T16:01:13.582Z', 'created_at': '2023-09-17T16:01:13.582Z', 'version': 1694966473582, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'BOOLEAN', 'name': 'Is Alcoholic', 'description': 'Enabling this toggle on an item indicates that it contains alcohol.', 'source_application': {'application_id': 'sq0idp-w46nJ_NCNDMSOywaCY0mwA', 'name': 'Square Online Store'}, 'allowed_object_types': ['ITEM'], 'seller_visibility': 'SELLER_VISIBILITY_HIDDEN', 'app_visibility': 'APP_VISIBILITY_HIDDEN', 'key': 'is_alcoholic'}}, {'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'KKJDEQ5ET5QY232B3XO4GRR2', 'updated_at': '2023-09-17T16:01:13.869Z', 'created_at': '2023-09-17T16:01:13.869Z', 'version': 1694966473869, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'STRING', 'name': 'Ecom Storefront Classi

In [181]:
pp.pprint(result.body)

{   'objects': [   {   'created_at': '2023-09-17T16:01:13.582Z',
                       'custom_attribute_definition_data': {   'allowed_object_types': [   'ITEM'],
                                                               'app_visibility': 'APP_VISIBILITY_HIDDEN',
                                                               'description': 'Enabling '
                                                                              'this '
                                                                              'toggle '
                                                                              'on '
                                                                              'an '
                                                                              'item '
                                                                              'indicates '
                                                                              'that '
                                  


## Query API, finds an object based upon characteristics

If you know the object ID, Retrieve Catalog Obj API in next section

### Query for a specific item by attribute (name)

In [66]:
result = sq_client.catalog.search_catalog_objects(
  body = {
    "query": {
      "exact_query": {
        "attribute_name": "name",
        "attribute_value": "Nick Fury Doll"
        #"attribute_value": "DR Doom Doll"
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'latest_time': '2023-10-01T14:05:39.93Z'}


In [67]:
pp.pprint(result.body)

{'latest_time': '2023-10-01T14:05:39.93Z'}


## Query for a specific item by attribute (abbreviation)

In [68]:
# I could not determine a way to query on the ID
# If you used attribute, you can specify the abbreviation but not the ID given after the ID>

result = sq_client.catalog.search_catalog_objects(
  body = {
    "object_types": [
      "ITEM"
    ],
    "query": {
      "exact_query": {
        "attribute_name": "abbreviation",
        "attribute_value": "NFDOLL"
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'latest_time': '2023-10-01T14:05:39.93Z'}


In [69]:
pp.pprint(result.body)

{'latest_time': '2023-10-01T14:05:39.93Z'}


## To read info about a particular item use "Retrieve Catalog Object"

In [50]:
# IDS                             Name               ABBR             Description                                  NOTES
# 44K7RNU7YHIXMYOV5X5TD2ZY        Nick Fury Doll     NFDOLL           something about Nick Fury from vertex ai
# FX3EJWXWG54ANMM32CNQDT3O        Dr Doom Doll       DRDOLL           something we want vertext ai to generate


In [59]:
# nick fury
object_id = 'FX3EJWXWG54ANMM32CNQDT3O'
# dr doom
#object_id = 'LUK7YOSRO4J6BV5SYVAU5X2P'
# overwirte for second dr doom doll added during demo

result = sq_client.catalog.retrieve_catalog_object(
  object_id = object_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'object': {'type': 'ITEM', 'id': 'FX3EJWXWG54ANMM32CNQDT3O', 'updated_at': '2023-09-30T14:14:50.836Z', 'created_at': '2023-09-30T14:14:08.045Z', 'version': 1696083290836, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Dr Doom Doll', 'description': 'something we want vertex ai to generate', 'abbreviation': 'DRDOLL', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'MWN7S42X45UBZPPVPFWJ4PYJ', 'updated_at': '2023-09-30T14:14:08.045Z', 'created_at': '2023-09-30T14:14:08.045Z', 'version': 1696083248045, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': 'FX3EJWXWG54ANMM32CNQDT3O', 'name': '6 inch poseable figure', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}], 'product_type': 'REGULAR', 'ecom_uri': 'https://rtp-gcp-usergroup.square.site/product/dr-doom-doll/24', 'ecom_image_uris': ['https://rtp-gcp-usergroup.squa

In [60]:
pp.pprint(result.body)

{   'object': {   'created_at': '2023-09-30T14:14:08.045Z',
                  'id': 'FX3EJWXWG54ANMM32CNQDT3O',
                  'is_deleted': False,
                  'item_data': {   'abbreviation': 'DRDOLL',
                                   'description': 'something we want vertex ai '
                                                  'to generate',
                                   'description_html': '<p>something we want '
                                                       'vertex ai to '
                                                       'generate</p>',
                                   'description_plaintext': 'something we want '
                                                            'vertex ai to '
                                                            'generate',
                                   'ecom_available': True,
                                   'ecom_image_uris': [   'https://rtp-gcp-usergroup.square.site/uploads/1/4/7/0/147028449/s933423469

# update the inventory count (This is a required api usage for the hackathon)

This will update the current count.  It is not an adjustment.

Example: if you currently have 10 items, you use this call to specify 15 items, afterwards inventory is 15 items.
There is another call which is "inventory adjustment" that will increase/decrease the inventory account 
by the specified amount.  For this reason, simply use the api to specify a count.

In [204]:
# Save off this variation id so we can use to update inventory

# print the id's of interest
print("action figure cat id: ", cat_obj_item_id)
print("dr doom var id: ", dr_doom_variation_id)
print("nick fury var id: ", nick_fury_variation_id)


action figure cat id:  7Z2R6MUWTYFGQ3K5PF7NL2WZ
dr doom var id:  FM7JRSIOFQ3D3BZGKM7BJZQG
nick fury var id:  2MD7GTL3C6H2KNWUNDJWKRJQ


# update inventory for Dr Doom Variation 

In [205]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

# constants between inventory updates
online_store_loc_id = "LZBNSDTSHAMM8"
quantity_update_number = "10"  # assume we want to specify a particular count, not adj by this amount.

# get current timestamp as ISO8601 yyyy-mm-ddThr:min:sec.sssZ
current_time = datetime.datetime.now().isoformat(timespec='seconds')
current_time = current_time + 'Z'
time_of_received_inventory = current_time


# Dr doom Inventory
variation_id = dr_doom_variation_id

result = sq_client.inventory.batch_change_inventory(
  body = {
    "idempotency_key": str(uniq_uuid),
    "changes": [
      {
        "type": "PHYSICAL_COUNT",
        "physical_count": {
          "catalog_object_id": variation_id,
          "state": "IN_STOCK",
          "location_id": online_store_loc_id,
          "quantity": quantity_update_number,
          "occurred_at": time_of_received_inventory
        }
      }
    ]
  }
)

# If the existing count is 15, and the update is to have
# 15, then the results will be {}.
#
# If the existing count is 10 and the update is 15,
# then the results will have some json

print("results:")
print(result)
if result.is_success():
  print("success")
  print(result.body)
elif result.is_error():
  print("failure")
  print(result.errors)

results:
<ApiResponse {"counts":[{"catalog_object_id":"FM7JRSIOFQ3D3BZGKM7BJZQG","catalog_object_type":"ITEM_VARIATION","state":"IN_STOCK","location_id":"LZBNSDTSHAMM8","quantity":"10","calculated_at":"2023-10-01T15:56:37.452Z"}]}>
success
{'counts': [{'catalog_object_id': 'FM7JRSIOFQ3D3BZGKM7BJZQG', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': 'LZBNSDTSHAMM8', 'quantity': '10', 'calculated_at': '2023-10-01T15:56:37.452Z'}]}


# update nick fury variation inventory count

In [206]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

# constants between inventory updates
online_store_loc_id = "LZBNSDTSHAMM8"
quantity_update_number = "5"  # assume we want to specify a particular count, not adj by this amount.

# get current timestamp as ISO8601 yyyy-mm-ddThr:min:sec.sssZ
current_time = datetime.datetime.now().isoformat(timespec='seconds')
current_time = current_time + 'Z'
time_of_received_inventory = current_time


# Nick Fury Inventory
variation_id = nick_fury_variation_id

result = sq_client.inventory.batch_change_inventory(
  body = {
    "idempotency_key": str(uniq_uuid),
    "changes": [
      {
        "type": "PHYSICAL_COUNT",
        "physical_count": {
          "catalog_object_id": variation_id,
          "state": "IN_STOCK",
          "location_id": online_store_loc_id,
          "quantity": quantity_update_number,
          "occurred_at": time_of_received_inventory
        }
      }
    ]
  }
)

# If the existing count is 15, and the update is to have
# 15, then the results will be {}.
#
# If the existing count is 10 and the update is 15,
# then the results will have some json

print("results:")
print(result)
if result.is_success():
  print("success")
  print(result.body)
elif result.is_error():
  print("failure")
  print(result.errors)

results:
<ApiResponse {"counts":[{"catalog_object_id":"2MD7GTL3C6H2KNWUNDJWKRJQ","catalog_object_type":"ITEM_VARIATION","state":"IN_STOCK","location_id":"LZBNSDTSHAMM8","quantity":"5","calculated_at":"2023-10-01T15:58:11.68Z"}]}>
success
{'counts': [{'catalog_object_id': '2MD7GTL3C6H2KNWUNDJWKRJQ', 'catalog_object_type': 'ITEM_VARIATION', 'state': 'IN_STOCK', 'location_id': 'LZBNSDTSHAMM8', 'quantity': '5', 'calculated_at': '2023-10-01T15:58:11.68Z'}]}


## Delete a catalog object item

`DONT RUN UNLESS YOU WANT TO RECREATE THE ITEM`

In [19]:
delete_id = "S5IJI4PUFA5VTB2Y6NDZ6HQV"
result = sq_client.catalog.delete_catalog_object(
  object_id = delete_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'deleted_object_ids': ['45KACZNXRA7PM32CAGXGRZA4', 'S5IJI4PUFA5VTB2Y6NDZ6HQV'], 'deleted_at': '2023-09-30T13:12:08.572Z'}
